In [ ]:
import requests
import gradio as gr 
import pandas as pd 
from PIL import Image, ImageDraw
import random
import time

eat_dataframe = pd.DataFrame(columns=["음식이름", "개수", "가격"])

def request_reciept(file_path) :

    with open(file_path, "rb") as image_to_read :
        image_data = image_to_read.read()
    
    endpoint = ""
    api_key = ""
    region = "northeurope"

    headers = {
        "Content-Type" : "image/jpg",
        "Ocp-Apim-Subscription-Key" : api_key
    }

    post_response = requests.post(endpoint, headers=headers, data=image_data)
    
    # Post response가 성공인 경우
    if post_response.status_code == 202 or post_response.status_code == 200 :
        result_url = post_response.headers['Operation-Location']
        
        # get response가 succeed 될때까지 while 반복
        while True :
            result_response = requests.get(result_url, headers={"Ocp-Apim-Subscription-Key" : api_key})
            result_response_json = result_response.json()        
            result_status = result_response_json['status']
            
            if result_status == 'running' :
                print(result_status)
                time.sleep(0.5)
                continue
            else :
                print(result_status)
                break

        if result_status == "succeeded" :
            item_list = result_response_json['analyzeResult']["documents"][0]["fields"]["Items"] 
                 
            return item_list
        # get response가 실패인 경우
        else :
            return print("Get Failed")
    # Post response가 실패인 경우
    else :
        return print("Post Failed")



In [ ]:

def request_speechtotext(file_path) :

    end_point_speech = ""
    api_key_speech = "" 

    headers_speech = {
        "Ocp-Apim-Subscription-Key" : api_key_speech,
        "Content-Type" : "audio/wav"
    }

    with open(file_path, "rb") as audio :
        audio_data = audio.read()

    response_audio = requests.post(end_point_speech, headers=headers_speech, data=audio_data)

    if response_audio.status_code == 200 :
        response_json = response_audio.json()
        text_result = response_json['DisplayText']
        
        return text_result
    else :
        return None



In [ ]:
def text_to_table(text) :

    end_point_language = ""
    api_key_language = ""
    
    headers_language = {
        "Content-Type" : "application/json",
        "Ocp-Apim-Subscription-Key" : api_key_language
    }

    body_language = {
    "kind": "EntityRecognition",
    "parameters": {
        "modelVersion": "latest"
    },
    "analysisInput":{
        "documents":[
            {
                "id":"1",
                "language": "ko",
                "text": text
            }
        ]
    }
}
    
    response_language = requests.post(end_point_language, headers=headers_language, json=body_language)
    response_language_json = response_language.json()
    result_language = response_language_json["results"]['documents'][0]["entities"]

    return result_language

In [ ]:
def random_color() :
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

In [ ]:
def draw_image(file_path, item_list) :
    image = Image.open(file_path)
    draw = ImageDraw.Draw(image)
    array_list = item_list["valueArray"]
    
    # 폴리곤 그리기 
    for array in array_list :        
        objects = array["valueObject"]
        for value in objects.values() :
            polygon = value["boundingRegions"][0]["polygon"]
            line_color = random_color()
            points = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]   
            draw.polygon(points, outline=line_color, width=1) 
    return image

In [ ]:
def convert_to_dataframe(item_list) :
    array_list = item_list["valueArray"]
    global eat_dataframe
    
    for array in array_list :  
        content = array["valueObject"]["Description"]["content"]
        quantity = int(array["valueObject"]["Quantity"]["content"])
        price = array["valueObject"]["TotalPrice"]["content"]
        eat_dataframe.loc[len(eat_dataframe)] = [content, quantity, price]

    return eat_dataframe

In [ ]:
def convert_to_dataframe_audio(result_language) :
    global eat_dataframe

    new_row = dict()
    for itemlist in result_language :        
        if itemlist["category"] == "Product" :
            name = itemlist["text"]
            new_row["음식이름"] = name
        elif itemlist["subcategory"] == "Number" :
            number = int(itemlist["text"])
            new_row["개수"] = number
        elif itemlist["subcategory"] == "Currency" :
            price = itemlist["text"]
            new_row["가격"] = price
        else :
            continue 

    new = pd.DataFrame([new_row])
    eat_dataframe = pd.concat([eat_dataframe, new], ignore_index=True)
    return eat_dataframe

In [ ]:
theme = gr.themes.Origin(
    primary_hue="rose",
    secondary_hue="red",
    neutral_hue="slate",
    text_size=gr.themes.Size(lg="17px", md="15px", sm="13px", xl="24px", xs="12px", xxl="28px", xxs="10px"),
    radius_size="lg",
    font=[gr.themes.GoogleFont('Gowun Batang'), gr.themes.GoogleFont('IBM Plex Sans KR '), gr.themes.GoogleFont('42dot Sans '), 'sans-serif'],
    font_mono=[gr.themes.GoogleFont('Gowun Batang'), gr.themes.GoogleFont('IBM Plex Sans KR '), gr.themes.GoogleFont('42dot Sans '), 'monospace'],
).set(
    body_background_fill='*background_fill_secondary',
    body_background_fill_dark='*neutral_800',    
    body_text_color='*neutral_700',
    body_text_size='*text_md',
    embed_radius='*radius_md',
    block_radius='*radius_md',
    block_title_radius='*radius_md',
    block_title_text_size='*text_md',
    container_radius='*radius_md',
    input_text_size='*text_sm',
    button_large_text_size='*text_md',
    form_gap_width='0px'     
)

with gr.Blocks(theme=theme) as demo :
    gr.Markdown("## 오늘 먹은 내역 정리하기 ", min_height="40px")
    with gr.Row() : 
        with gr.Column() :
            gr.Markdown("### 영수증 입력하기", min_height="25px")
            image_upload = gr.Image(label="영수증 업로드", sources=["upload","clipboard"], type="filepath", width="400px")
            img_send_btn = gr.Button("전송")
        with gr.Column() :
            gr.Markdown("### 영수증 분석결과", min_height="25px")
            image_result = gr.Image(label="분석 결과", width="400px")
        with gr.Column() :
            gr.Markdown("### 영수증이 없다면 말로 입력하기", min_height="25px")
            audio_input = gr.Audio(sources="microphone", type="filepath")
            audio_send_btn = gr.Button("전송")
            audio_output = gr.Textbox(label="분석 결과")
            clear_btn = gr.Button("전체 지우기")  
    gr.Markdown("### 오늘 먹은 내역 정리 Table", min_height="25px")
    eat_result = gr.Dataframe()
        
    def img_click(file_path) :
        item_list = request_reciept(file_path)
        drawed_image = draw_image(file_path, item_list)
        converted_data = convert_to_dataframe(item_list)
        return drawed_image, converted_data
    
    def audio_click(file_path) :
        text = request_speechtotext(file_path)
        result_language = text_to_table(text)
        converted_data = convert_to_dataframe_audio(result_language) 
        return  text, converted_data
    
    def clean() :
        global eat_dataframe
        eat_dataframe = eat_dataframe.drop(eat_dataframe.index)
        return None, None, None, None
     
    img_send_btn.click(img_click, inputs=[image_upload], outputs=[image_result, eat_result])
    # image_upload.change(img_click, inputs=[image_upload], outputs=[image_result, eat_result])
    audio_send_btn.click(audio_click, inputs=[audio_input], outputs=[audio_output, eat_result])
    clear_btn.click(clean, inputs=[], outputs=[image_upload, image_result, audio_input, audio_output])

if __name__ == "__main__":
    demo.launch()  